<h1>Multiview Registration RANSAC</h1>

In [97]:
from registration_funcs import *

VOXEL_SIZE = 40 # 20 = normal, 15 = final

Read raw point clouds

In [98]:
pcd_A = o3d.io.read_point_cloud('./view_1_rgb_pcd_light_obj_dectection_colour.ply')
pcd_B = o3d.io.read_point_cloud('./view_2_rgb_pcd_light_obj_dectection_colour.ply')

print(number_of_points(pcd_A))
print(number_of_points(pcd_B))

51853
58858


Downsample raw point clouds

In [99]:
pcd_A_ds = pcd_A.voxel_down_sample(voxel_size=VOXEL_SIZE)
pcd_B_ds = pcd_B.voxel_down_sample(voxel_size=VOXEL_SIZE)

print(number_of_points(pcd_A_ds))
print(number_of_points(pcd_B_ds))

859
635


Seperate object and background using DBSCAN

In [100]:
clusters, aabbs = get_clusters(pcd_A_ds)
pcd_A_crp = pcd_A_ds.crop(aabbs[1])

clusters, aabbs = get_clusters(pcd_B_ds)
pcd_B_crp = pcd_B_ds.crop(aabbs[0])

o3d.visualization.draw_geometries([pcd_A_crp, pcd_B_crp])

Disturb inital pose and calculate FPFH feature points

In [101]:
source, target, source_down, target_down, source_fpfh, target_fpfh = prepare_dataset(pcd_A_crp, pcd_B_crp, VOXEL_SIZE)

:: Load two point clouds and disturb initial pose.
:: Estimate normal with search radius 80.000.
:: Compute FPFH feature with search radius 200.000.
:: Estimate normal with search radius 80.000.
:: Compute FPFH feature with search radius 200.000.


Compute transform using RANSAC

In [102]:
result_ransac = execute_global_registration(source_down, target_down, source_fpfh, target_fpfh, VOXEL_SIZE)
print(result_ransac)

:: RANSAC registration on downsampled point clouds.
   Since the downsampling voxel size is 40.000,
   we use a liberal distance threshold 60.000.
[Open3D WARNING] Too few correspondences (39) after mutual filter, fall back to original correspondences.
RegistrationResult with fitness=7.482185e-01, inlier_rmse=3.429813e+01, and correspondence_set size of 315
Access transformation to get result.


Refine results using ICP

In [103]:
result_icp = refine_registration(source, target, source_fpfh, target_fpfh, result_ransac, VOXEL_SIZE)
print(result_icp)

:: Point-to-plane ICP registration is applied on original point
   clouds to refine the alignment. This time we use a strict
   distance threshold 16.000.
RegistrationResult with fitness=2.897862e-01, inlier_rmse=1.144847e+01, and correspondence_set size of 122
Access transformation to get result.


Display final result

In [105]:
output_pcd = get_registration_result(source, target, result_icp.transformation)
o3d.visualization.draw_geometries([output_pcd])
print(number_of_points(output_pcd))

764
